# DePlot: One-shot visual language reasoning by plot-to-table translation

### 1. どんなもの？
one-shot モデルのDeplotの紹介

### 2. 先行研究と比べるとどこが凄い？  
* 今まではend2endモデルだったが、(1)画像からグラフや図の情報を抽出してlinearized tableに変換する, (2)LLMに入力して答えを得る.という二段階のモデルにした。End2endモデルを訓練するには大量のデータと計算が必要であり、それでもなお複雑なQuestionについては欠陥があった。Deplotは様々なタイプのタスクにも対応することが出来る。
* 新たなchart2tableの評価指標であるRMS, which stands for Relative Mapping Similarityを提案. いままでに使われてきた評価指標であるRNSS, which stands for relative number set similarityは表の中の数字の位置を考慮しないものであった。


### 3. 技術や手法のキモはどこ？
* Deplot  
  image-to-text encoder-decoder TransformerをSOTAだったMaTCHAのweightに初期化してそこから訓練を始める。tableはマークダウン形式を取っており、セルが別であることを"|", 列が離れていることを"\n"で表現する。
* LLMのPrompt  
  Deplotの出力と質問文を入力する。Chain-of-thoghtで行う。(問に対する答えを出す段階の「思考過程」を学習させる)また、Self-consistencyでも実験を行った。
  > [Self-ConsistencyはCoTと同様に思考ステップをPromptの中に例として含めます。Self-Consistencyではさらに1つのPromptに対して複数の回答をLLMに生成させます。](https://techblog.cccmk.co.jp/entry/2023/04/04/102443)
* RMS
  * 行や列のように置き換えに不変であること、数字や文字の小さなエラーはある程度まで許しpenelizeすること、適合率や再現率の損失を明確に反映させることを考慮して以下の通りに設定する.ただし$r$: row, $c$: columnの値を$v$とする.    
  (補足)(1)文章の入力間を適切に測るには Normalized Levenshtein Distance, or $NL_{\tau}$が使われる.(2)$D_{\theta}^{p, t}$は1に近いほど類似しており、0に近いほど乖離していることを示す.
  
  $$
  p_{i} = (p_{i}^{r}, p_{i}^{c}, p_{i}^{v}) \\
  t_{j} = (t_{j}^{r}, t_{j}^{c}, t_{j}^{v}) \\
  D_{\tau, \theta} = (1-NL_{\tau}(p^{r}||p^{c}, t^{r}||t^{c}))(1-D_{\theta}(p^{v}, t^{v})) \\
  \\
  RMS_{precision} = 1 - \frac{\sum_{i=1}^{N}\sum_{j=1}^{M}X_{ij}D_{\tau, \theta}(p_{i}, t_{j})}{N} \\
  RMS_{recall} = 1 - \frac{\sum_{i=1}^{N}\sum_{j=1}^{M}X_{ij}D_{\tau, \theta}(p_{i}, t_{j})}{M}
  $$

### 4. どうやって有効だと検証した？
* 手書きChartQAタスクでSOTA達成(24.0%改善)
* ChartQA, PlotQA

### 5. 議論はある？
* LLMを数値計算やプログラミングに特化させたものにしたらどうなるのだろうか？
* chart-to-tableの段階で色や形などの情報が失われてしまうので図の属性を表したものを出力させることを考えないといけない。

### 6. 次に読むべき論文は？
* ChartOCR
* Chain-of-thoght
* Self-consistency

### 7. 実装
No official code found